In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, wandb, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login



In [24]:
base_model =  "mistralai/Mistral-7B-v0.1" #"filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
dataset_name, new_model = "AdrienB134/ASN_pairs", "AdrienB134/ASN_mistral_7b"

In [56]:
dataset = load_dataset(dataset_name, split="train")
dataset['text'][0]

"[INST] Quelle est la date de l'inspection inopinée réalisée par EDF/CEIDRE au CNPE de Nogent ? [/INST] L'inspection inopinée a eu lieu le 15 février 2010."

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    padding = True,
    truncation=True
    )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(True, True)

In [61]:

from google.colab import userdata
key = userdata.get('WANB')

wandb.login(key = key)
run = wandb.init(project='Fine tuning mistral 7B COLAB', job_type="training", anonymous="allow")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train/epoch,▁▁▁▂▂▂▂▂▃▃▃▁▁▁▂▂▂▂▂▃▃▃▃▄▄▁▁▂▂▃▃▄▄▅▅▆▇▇██
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▁▁▁▂▂▂▂▂▃▃▃▃▄▄▁▁▂▂▃▃▄▄▅▆▆▇▇██
train/learning_rate,█▇▇▆▅▅▄▃▃▂▁█▇▇▆▆▅▅▄▄▃▃▂▂▁██▇▇▆▆▅▅▄▄▃▃▂▂▁
train/loss,█▆▅▃▄▄▄▄▄▃▄▇▇▅▂▄▇▄▂▅▂▁▄▄▃▇▅▅▄▃▄▃▃▂▃▃▃▂▃▂
train/total_flos,▁▂█
train/train_loss,█▆▁
train/train_runtime,▁▂█
train/train_samples_per_second,▂▁█
train/train_steps_per_second,▂▁█
train/epoch,0.87
train/global_step,1000


In [62]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

In [ ]:
import transformers
tokenized_train_dataset = dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir=new_model,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=3000,
        learning_rate=2.5e-4,
        weight_decay= 0.001,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_steps=25,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        #save_strategy="steps",       # Save the model checkpoint every logging step
        #save_steps=25,                # Save checkpoints every 50 steps
        #evaluation_strategy="steps", # Evaluate the model every logging step
        #eval_steps=25,               # Evaluate and save checkpoints every 50 steps
        #do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name= "test-teansformers-400steps" # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
wandb.finish()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.934900
50,1.813400
75,1.683200
100,1.508700
125,1.600700
150,1.562500
175,1.843300
200,1.441300
225,1.436400
250,1.591200


Step,Training Loss
25,1.934900
50,1.813400
75,1.683200
100,1.508700
125,1.600700
150,1.562500
175,1.843300
200,1.441300
225,1.436400
250,1.591200


Checkpoint destination directory AdrienB134/ASN_mistral_7b/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Let's try to train using SFT

In [8]:

training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 1,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 5000,
    logging_steps= 10,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= 90,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
    report_to="wandb"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2307 [00:00<?, ? examples/s]

In [9]:
trainer.train()
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,2.304300
60,1.340100
90,0.890400


train/epoch,▁▅██
train/global_step,▁▅██
train/learning_rate,▁▁▁
train/loss,█▃▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,0.62
train/global_step,90


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer):

In [10]:
model.config.use_cache = True

def stream(user_prompt):
    runtimeFlag = "cuda:0"
    #system_prompt = "Tu es une IA du nucléaire, tu réponds aux questions et tu t'arrêtes. \n"
    B_INST, E_INST = "[INST]", "[/INST]"

    prompt = f"{B_INST}{user_prompt.strip()}\n{E_INST}"

    inputs = tokenizer([user_prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=200)

In [13]:
stream("Quel est le délai donné par l'ASN au CNPE de Penly pour répondre aux demandes formulées suite à l'inspection ?")

NameError: name 'tokenizer' is not defined

In [12]:
# Clear the memory footprint
del model, trainer, tokenizer
torch.cuda.empty_cache()